In [65]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
from geojson import Feature, FeatureCollection, Point
from datetime import datetime, timedelta
import re
import os

## Get desired date variables

In [66]:
week_start = '2022-6-11'
week_end = '2022-12-11'

week_start = datetime.strptime(week_start, '%Y-%d-%m')
week_end = datetime.strptime(week_end, '%Y-%d-%m')

## Read-in Data

In [67]:
df = pd.read_csv('redfin_2022-11-15-14-49-21.csv', dtype={'YEAR BUILT':'str'})

In [68]:
len(df)

228

In [69]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [70]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [71]:
df = df[(df['SOLD DATE'] >= week_start) & (df['SOLD DATE'] <= week_end)]

In [72]:
len(df)

36

In [73]:
df = df.sort_values(by='PRICE', ascending=False).head(10)
df['PRICE'] = df['PRICE'].apply(lambda x: '${:,}'.format(x))
df['$/SQUARE FEET'] = df['$/SQUARE FEET'].apply(lambda x: '${:,}'.format(x))

## Correction Section

## Map making section

In [74]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [79]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(), zoom_start=12)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Top 10 MLS-register Cook County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 6th - November 12th')

for i,r in df.iterrows():
    location = (r["LATITUDE"], r["LONGITUDE"])
    info = (r["PRICE"], r['$/SQUARE FEET'], r['ADDRESS'], r['YEAR BUILT'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[-2]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}\
    Price sq ft: {bold_start}{info[1]}{bold_end}{new_line} \
    Year Built: {bold_start}{info[-1]}{bold_end}{new_line}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [76]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [77]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [78]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chi_weekly_condo_sales_11_7_11_11_2022
